In [ ]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient
import datetime

In [ ]:
#Path for the local host. No need to change this

MONGO_HOST= 'mongodb://localhost/twitterdb' 



################# This is where you put your keywords in, feel free to add as much as you want. Make sure they are under the quotation marks.
WORDS = ['workathome', '#covid', '#datascience', '#staysafe']


# This is where I specify what language the Tweets should come in.
language = ['en']



################################## INSERT THE KEYS HERE  ############################################################

CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""

####################################################################################################



class StreamListener(tweepy.StreamListener):    
    #This is a class provided by tweepy to access the Twitter Streaming API. 

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("\nYou are now connected to Project Twitter. \n")
 
    def on_error(self, status_code):
        # On error - if an error occurs, display the error / status code
        print('An Error has occured: ' + repr(status_code))
        return False
 
    def on_data(self, data):
        #This is the main script
        try:
            client = MongoClient(MONGO_HOST)
            
            
############# Please name the database after where it says client.  If it doesn't exist, it will be created.################
            db = client.twitter_final
###################################################################################################            
    
    
            # Decode the JSON from Twitter
            datajson = json.loads(data)
            
            #grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']
    
            #print out a message to the screen that we have collected a tweet
            print("Tweet collected at " + str(created_at))
            
################# Please name the Collection after where it says db. If it doesn't exist, it will be automatically created #######################################
            db.tweeets.insert(datajson)
######################################################################################        
        except Exception as e:
            print(e)
            
            
            
            
#####################################  Leave this code as it is #############################################
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

#Set up the listener. The 'wait_on_rate_limit=True' is needed to help with Twitter API rate limiting.

listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True)) 
streamer = tweepy.Stream(auth=auth, listener=listener)
print("Gathering the specific hashtags: " + str(WORDS))
streamer.filter(track=WORDS, languages=language)